## Import packages

In [2]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import seaborn as sns
import pandas as pd
import pickle
import scanpy as sc
sc.set_figure_params(dpi=100, dpi_save=300)
import scvi
import anndata as ad
from matplotlib import pyplot, cm
import os
from math import ceil
from scipy.stats import spearmanr
import math

import leidenalg

from anndata import AnnData
import scanpy as sc
from scanpy import read
import pandas as pd
from sciPENN.Preprocessing import preprocess

import matplotlib.pyplot as plt
print(scvi.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0.20.3


# Read data: Pbmc (train), H1N1 (test)

In [3]:
adata_gene = sc.read("../Data/pbmc/pbmc_gene.h5ad")
adata_protein = sc.read("../Data/pbmc/pbmc_protein.h5ad")

In [4]:
adata_malt_gene = sc.read_10x_h5("../Data/malt_10k_protein_v3_filtered_feature_bc_matrix.h5")
adata_malt = sc.read("../Data/filtered_feature_bc_matrix/matrix.mtx").T
malt_features =  pd.read_csv("../Data/filtered_feature_bc_matrix/features.tsv", sep="\t", header=None)

adata_malt.var["feature_type"] = list(malt_features[2])
adata_malt.obs_names = adata_malt_gene.obs_names
adata_malt.var['protein_names'] = list(malt_features[0])
adata_malt.var_names = list(malt_features[0])

adata_malt_protein = adata_malt[:,adata_malt.var['feature_type'] == 'Antibody Capture']

adata_malt_gene.var_names_make_unique()

adata_gene_test = adata_malt_gene.copy()

adata_protein_test = adata_malt_protein.copy()
adata_gene_test.obs['sample'] = [1]*8412
adata_protein_test.obs['sample'] = [1]*8412
adata_protein_test

C:\Users\26074\.conda\envs\TotalVI\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
C:\Users\26074\.conda\envs\TotalVI\lib\site-packages\anndata\_core\anndata.py:1840: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 8412 × 17
    obs: 'sample'
    var: 'feature_type', 'protein_names'

In [5]:
common_genes = np.intersect1d(adata_gene.var.index, adata_gene_test.var.index)
common_proteins = np.intersect1d(adata_protein.var.index, adata_protein_test.var.index)

In [6]:
adata_malt_protein

View of AnnData object with n_obs × n_vars = 8412 × 17
    var: 'feature_type', 'protein_names'

# Selecting highly variable genes - using gene expression measures from test data 

In [7]:
gene_train, protein_train, gene_test, bools, train_keys, categories = preprocess([adata_gene], [adata_protein], adata_gene_test, train_batchkeys = ["donor"], gene_list = [], select_hvg = True, cell_normalize = True, log_normalize = True, gene_normalize = True, min_cells = 30, min_genes = 200)


QC Filtering Training Cells
QC Filtering Testing Cells

QC Filtering Training Genes
QC Filtering Testing Genes

Normalizing Training Cells
Normalizing Testing Cells

Log-Normalizing Training Data
Log-Normalizing Testing Data

Finding HVGs


C:\Users\26074\.conda\envs\TotalVI\lib\site-packages\anndata\_core\anndata.py:1763: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(



Normalizing Gene Training Data by Batch


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:06<00:00,  1.15it/s]



Normalizing Protein Training Data by Batch


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:03<00:00,  2.59it/s]



Normalizing Gene Testing Data by Batch


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.71it/s]


In [8]:
hvg = gene_test.var_names
cells_test = gene_test.obs_names
cells_train = gene_train.obs_names

In [9]:
del gene_train
del gene_test
del protein_train

import gc
gc.collect()

7133

# Format data

In [10]:
# What proteins overlap between the test and train data?

train_protein = adata_protein.var_names
test_protein = adata_protein_test.var_names
overlap_protein = train_protein[train_protein.isin(test_protein)]

In [11]:
## Subsetting the data by the HVG - pbmc

adata_gene_pbmc_hvg = adata_gene[cells_train, hvg].copy()

In [12]:

## Subsetting the data by the HVG - h1n1

adata_gene_h1n1_hvg = adata_gene_test[cells_test, hvg].copy()

In [13]:
(adata_gene_pbmc_hvg.var.index == adata_gene_h1n1_hvg.var.index).mean()

1.0

In [14]:
adata_protein = adata_protein[cells_train, :].copy()
adata_protein_test = adata_protein_test[cells_test, :].copy()

In [15]:
# Batches (subject) in training data - pbmc (8 subjects)

adata_gene_pbmc_hvg.obs['patient'] = pd.DataFrame(adata_gene_pbmc_hvg.obs['donor']).copy()
adata_gene_pbmc_hvg.obs['patient'] = adata_gene_pbmc_hvg.obs['donor'].astype("str")

In [16]:
# Batches (subject) in test data - h1n1 (20 subjects)

adata_gene_h1n1_hvg.obs['patient'] = pd.DataFrame(adata_gene_h1n1_hvg.obs['sample']).copy()
adata_gene_h1n1_hvg.obs['patient'] = adata_gene_h1n1_hvg.obs['patient'].astype("str")

In [17]:
## Combine data

adata = ad.concat([adata_gene_pbmc_hvg.copy(), adata_gene_h1n1_hvg.copy()],
                     join='outer')

### Note: Train on PBMC

In [18]:
train_patients = adata.obs["patient"].unique()[0:8]

In [19]:
test_patients = adata.obs["patient"].unique()[8:]

# Subset data based on HVGs and Hold Out Test Protein Set

In [20]:
adata_final = adata.copy()

In [21]:
held_out_proteins = adata_protein_test[cells_test, overlap_protein].copy()

### Now we hold-out the proteins for the test patients dataset. To do so, we can replace all the values with 0s. We will store the original values to validate after training.

In [22]:
# Modified this code cell to predict all p = 224 proteins

n, p = adata_protein.shape
n_H1N1, p_H1N1 = adata_protein_test.shape

protein_dat = pd.DataFrame(np.zeros(shape = (n + n_H1N1, p), dtype = 'float32'), 
                           index = list(adata_protein.obs_names) + list(adata_protein_test.obs_names),
                           columns = adata_protein.var_names)

protein_dat.iloc[:n] = adata_protein.X.toarray().copy() #fill the protein training data, leave test data as 0s

adata_final.obsm["protein_expression"] = protein_dat


# Remove additional data from memory:

In [23]:
del adata_gene
del adata_protein
del adata_protein_test
del adata_gene_test
del adata_gene_pbmc_hvg
del adata

# Run TotalVI

In [24]:
scvi.model.TOTALVI.setup_anndata(adata_final, batch_key="patient", 
                        protein_expression_obsm_key="protein_expression")

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


INFO     Using column names from columns of adata.obsm['protein_expression']                                       
INFO     Found batches with missing protein expression                                                             


In [26]:
totalvae = scvi.model.TOTALVI(
    adata_final,
    latent_distribution = "normal",
    n_layers_decoder = 2)

INFO     Computing empirical prior initialization for protein background.                                          


C:\Users\26074\.conda\envs\TotalVI\lib\site-packages\scvi\model\_totalvi.py:130: UserWarning: Some proteins have all 0 counts in some batches. These proteins will be treated as missing measurements; however, this can occur due to experimental design/biology. Reinitialize the model with `override_missing_proteins=True`,to override this behavior.
  warnings.warn(msg, UserWarning)


In [27]:
totalvae.view_anndata_setup(adata_final)

Anndata setup with scvi-tools version 0.20.3.

Setup via `TOTALVI.setup_anndata` with arguments:

{
│   'protein_expression_obsm_key': 'protein_expression',
│   'protein_names_uns_key': None,
│   'batch_key': 'patient',
│   'layer': None,
│   'size_factor_key': None,
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None
}

         Summary Statistics          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     Summary Stat Key     ┃ Value  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│         n_batch          │   9    │
│         n_cells          │ 170133 │
│ n_extra_categorical_covs │   0    │
│ n_extra_continuous_covs  │   0    │
│         n_labels         │   1    │
│        n_proteins        │  224   │
│          n_vars          │  1000  │
└──────────────────────────┴────────┘

                   Data Registry                   
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Registry Key ┃       scvi-tools Location        ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      X       │             adata.X              │
│    batch     │     adata.obs['_scvi_batch']     │
│    labels    │    adata.obs['_scvi_labels']     │
│   proteins   │ adata.obsm['protein_expression'] │
└──────────────┴──────────────────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

                   batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃   Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['patient'] │     1      │          0          │
│                      │     P1     │          1          │
│                      │     P2     │          2          │
│                      │     P3     │          3          │
│                      │     P4     │          4          │
│                      │     P5     │          5          │
│                      │     P6     │          6          │
│                      │     P7     │          7          │
│                      │     P8     │          8          │
└──────────────────────┴────────────┴─────────────────────┘

In [28]:
# Training with the default number of epochs 

# Training with the default number of epochs 
n_epochs = 40
lr = 4e-3

if os.path.isdir('weights_dir/totalvi_seurattomalt'):
    totalvae = totalvae.load("weights_dir/totalvi_seurattomalt", adata = adata_final)
else:
    totalvae.train(max_epochs=400)
    plt.plot(totalvae.history["elbo_validation"], label="test")
    plt.title("Negative ELBO over training epochs")
    plt.legend()
    
    totalvae.save("weights_dir/totalvi_seurattomalt")

INFO     File weights_dir/totalvi_seurattomalt\model.pt already downloaded                                         
INFO     Found batches with missing protein expression                                                             
INFO     Computing empirical prior initialization for protein background.                                          


C:\Users\26074\.conda\envs\TotalVI\lib\site-packages\scvi\model\_totalvi.py:130: UserWarning: Some proteins have all 0 counts in some batches. These proteins will be treated as missing measurements; however, this can occur due to experimental design/biology. Reinitialize the model with `override_missing_proteins=True`,to override this behavior.
  warnings.warn(msg, UserWarning)


# Analyze output - Results on training data

In [29]:
_, protein_means = totalvae.get_normalized_expression(
    transform_batch=train_patients,
    include_protein_background=True,
    sample_protein_mixing=False,
    return_mean=True,
)

#### Note that: transform_batch is a power parameter. Setting this allows one to predict the expression of cells as if they came from the inputted batch. In this case, we’ve observed protein expression in the training batchs “RPM211 and RPM232” (batch categories from original adata object), but we have no protein expression in the test batchs “RPM215 and RPM218”. We’d like to take the cells of the trainig batch and make a counterfactual prediction: “What would the expression look like if my batch "RPM211 and RPM232" cells came from batch “RPM215 and RPM218”?”

In [30]:
X_totalVI = pd.DataFrame(totalvae.get_latent_representation(), index = adata_final.obs.index)
X_totalVI.to_csv("totalvi_maltembedding.csv")

# Imputed protein expression: 

In [31]:
true_protein_test = pd.DataFrame(held_out_proteins.X.toarray(), index = held_out_proteins.obs.index, columns = held_out_proteins.var.index)

In [32]:
imputed_pros = protein_means[adata_final.obs.patient.isin(test_patients)]

pat_names = adata_final.obs['patient'].isin(test_patients)
patients = adata_final.obs.patient[pat_names].values
imputed_proteins_test = imputed_pros.copy()
# imputed_proteins_test = imputed_pros[overlap_protein] # Subset totalvi output to only include overlapping proteins

In [33]:
def corr2_coeff(A, B, pearson = True):
    if pearson:
        # Rowwise mean of input arrays & subtract from input arrays themeselves
        A_mA = A - A.mean(1)[:, None]
        B_mB = B - B.mean(1)[:, None]

        # Sum of squares across rows
        ssA = (A_mA**2).sum(1)
        ssB = (B_mB**2).sum(1)

        # Finally get corr coeff
        corr_mat = np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
        
        return corr_mat[range(corr_mat.shape[0]), range(corr_mat.shape[0])]
    
    else:
        corrs = [0.] * A.shape[0]
        
        for i in range(A.shape[0]):
            corrs[i] = spearmanr(A[i], B[i])[0]
            
        return corrs

In [34]:
# Normalize totalvi output, and gold standard counts

true_protein_test = AnnData(true_protein_test)
imputed_proteins_test = AnnData(imputed_proteins_test)

sc.pp.normalize_total(true_protein_test)
sc.pp.log1p(true_protein_test)

sc.pp.normalize_total(imputed_proteins_test)
sc.pp.log1p(imputed_proteins_test)

for patient in test_patients:
    indices = [x == patient for x in patients]
    sub_adata = imputed_proteins_test[indices]
    sc.pp.scale(sub_adata)
    imputed_proteins_test[indices] = sub_adata.X
    
    sub_adata = true_protein_test[indices]
    sc.pp.scale(sub_adata)
    true_protein_test[indices] = sub_adata.X

true_protein_test = pd.DataFrame(true_protein_test.X, columns = true_protein_test.var.index)
imputed_proteins_test = pd.DataFrame(imputed_proteins_test[:, overlap_protein].X, columns = overlap_protein)

C:\Users\26074\.conda\envs\TotalVI\lib\site-packages\scanpy\preprocessing\_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
C:\Users\26074\.conda\envs\TotalVI\lib\site-packages\scanpy\preprocessing\_simple.py:843: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [35]:
sq = lambda x, y: (x - y)**2

In [36]:
corrs_table = np.zeros((imputed_proteins_test.shape[1], len(np.unique(patients))))
sq_table = corrs_table.copy()

for i, patient in enumerate(np.unique(patients)):
    truth = true_protein_test[patients == patient].to_numpy()
    imputed = imputed_proteins_test[patients == patient].to_numpy()

    corrs_table[:, i] = corr2_coeff(truth.T, imputed.T)
    sq_table[:, i] = sq(truth, imputed).mean(axis = 0)

if np.isnan(corrs_table).sum() > 0:
    corrs_table[np.isnan(corrs_table)] = 0
    
corrs_table = pd.DataFrame(corrs_table)
sq_table = pd.DataFrame(sq_table)
corrs_table.index, corrs_table.columns = imputed_proteins_test.columns, np.unique(patients)
sq_table.index, sq_table.columns = imputed_proteins_test.columns, np.unique(patients)

In [37]:
#here are correlations

corrs_table

,1
index,
CD19,0.166665
CD45RA,0.230537
CD8a,0.165554
CD14,0.139048
CD25,-0.323961
CD45RO,0.402813
TIGIT,0.127485
CD127,0.407962
CD15,0.041121


In [38]:
#here are correlations

corrs_table.mean().mean()

0.13145494982600212

In [39]:
corrs_table.to_csv('../corrs_results/totalvi_malt.csv')

In [40]:
sq_table

,1
index,
CD19,1.666471
CD45RA,1.538743
CD8a,1.668692
CD14,1.721699
CD25,2.647607
CD45RO,1.194232
TIGIT,1.744821
CD127,1.183935
CD15,1.917530


In [41]:
sq_table.mean().mean()

1.7368829369544982

In [42]:
sq_table.to_csv('../mse_results/totalvi_malt.csv')

In [43]:
imputed_proteins_test.to_csv('totalvi_maltfeatures.csv')

In [44]:
_, protein_means_samples = totalvae.get_normalized_expression(
    transform_batch=train_patients,
    n_samples=100,
    include_protein_background=True,
    sample_protein_mixing=False,
    return_mean=False,
    indices = list(range(n, len(protein_dat)))
)

In [45]:
name_map = {protein: index for index, protein in enumerate(imputed_pros.columns)}
index_overlap = [name_map[protein] for protein in overlap_protein]

In [46]:
protein_means_samples_norm = np.empty(protein_means_samples.shape)

imputed_pros = protein_means[adata_final.obs.patient.isin(test_patients)]

# normalize totalvi sample output
sf = protein_means_samples.sum(axis = 1)
sf = np.median(sf, axis = 0)[None, :]/sf

protein_means_samples_norm = protein_means_samples * sf[:, None, :]
protein_means_samples_norm = np.log(protein_means_samples_norm + 1)
protein_means_samples_norm = protein_means_samples_norm[:, index_overlap]

for patient in test_patients:
    indices = [x == patient for x in patients]
    sub_data = protein_means_samples_norm[indices]
    mean, sd = sub_data.mean(axis = 0), sub_data.std(axis = 0)
    sub_data = (sub_data - mean)/sd
    protein_means_samples_norm[indices] = sub_data

In [47]:
protein_means_samples_norm.shape

(8385, 10, 100)

In [48]:
q10 = np.percentile(protein_means_samples_norm, 10, axis = 2)
q25 = np.percentile(protein_means_samples_norm, 25, axis = 2)
q75 = np.percentile(protein_means_samples_norm, 75, axis = 2)
q90 = np.percentile(protein_means_samples_norm, 90, axis = 2)

In [49]:
q90.shape

(8385, 10)

In [50]:
cols = imputed_pros[overlap_protein].columns
names = imputed_pros[overlap_protein].index
q10 = pd.DataFrame(q10,  columns=cols)
q90 = pd.DataFrame(q90,  columns=cols)
q25 = pd.DataFrame(q25,  columns=cols)
q75 = pd.DataFrame(q75, columns=cols)

In [53]:
base_path = "totalVI_quantiles_malt"

In [57]:
os.getcwd()

'D:\\Papercode\\sciPENN_codes-master\\sciPENN_codes-master\\Experiments'

In [62]:
true_protein_test.to_csv(os.path.join(base_path, "truth.csv"))

In [54]:
r50 = (true_protein_test < q75)
l50 = (true_protein_test > q25)

print(f"Effective Coverage Probability for Nominal 50% PIs: {(r50*l50).mean()}")
print(f"Mean effective Coverage Probability for Nominal 50% PI: {(r50*l50).mean().mean()}")

Effective Coverage Probability for Nominal 50% PIs: CD19      0.004532
CD45RA    0.005844
CD8a      0.004293
CD14      0.005247
CD25      0.003101
CD45RO    0.004532
TIGIT     0.003220
CD127     0.003697
CD15      0.005486
CD16      0.004293
dtype: float64
Mean effective Coverage Probability for Nominal 50% PI: 0.004424567680381635


In [55]:
r80 = (true_protein_test < q90)
l80 = (true_protein_test > q10)

print(f"Effective Coverage Probability for Nominal 80% PIs: {(r80*l80).mean()}")
print(f"Mean effective Coverage Probability for Nominal 80% PI: {(r80*l80).mean().mean()}")

Effective Coverage Probability for Nominal 80% PIs: CD19      0.007990
CD45RA    0.009779
CD8a      0.011807
CD14      0.011449
CD25      0.006202
CD45RO    0.008945
TIGIT     0.006559
CD127     0.006917
CD15      0.010733
CD16      0.007752
dtype: float64
Mean effective Coverage Probability for Nominal 80% PI: 0.008813357185450208


In [58]:
base_path = 'totalVI_quantiles_malt'

if not os.path.isdir(base_path):
    os.mkdir(base_path)

In [59]:
q10.to_csv(os.path.join(base_path, 'q10.csv'))
q25.to_csv(os.path.join(base_path, 'q25.csv'))
q75.to_csv(os.path.join(base_path, 'q75.csv'))
q90.to_csv(os.path.join(base_path, 'q90.csv'))

<module 'ntpath' from 'C:\\Users\\26074\\.conda\\envs\\TotalVI\\lib\\ntpath.py'>